In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/kaggle'

In [2]:
!mkdir kaggle/jigsaw kaggle/impermium
%cd ./kaggle/jigsaw
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge
!unzip train.csv.zip
!unzip test.csv.zip
!unzip test_labels.csv.zip

/content/kaggle/jigsaw
 51% 12.0M/23.4M [00:00<00:00, 124MB/s]
100% 23.4M/23.4M [00:00<00:00, 115MB/s]
  0% 0.00/1.39M [00:00<?, ?B/s]
100% 1.39M/1.39M [00:00<00:00, 93.7MB/s]
 38% 10.0M/26.3M [00:00<00:00, 102MB/s]
100% 26.3M/26.3M [00:00<00:00, 128MB/s]
  0% 0.00/1.46M [00:00<?, ?B/s]
100% 1.46M/1.46M [00:00<00:00, 208MB/s]
Archive:  train.csv.zip
  inflating: train.csv               
Archive:  test.csv.zip
  inflating: test.csv                
Archive:  test_labels.csv.zip
  inflating: test_labels.csv         


In [3]:
%cd ../impermium
!kaggle competitions download -c detecting-insults-in-social-commentary
%cd ../..

/content/kaggle/impermium
  0% 0.00/831k [00:00<?, ?B/s]
100% 831k/831k [00:00<00:00, 54.8MB/s]
  0% 0.00/294k [00:00<?, ?B/s]
100% 294k/294k [00:00<00:00, 92.2MB/s]
  0% 0.00/604k [00:00<?, ?B/s]
100% 604k/604k [00:00<00:00, 193MB/s]
  0% 0.00/323k [00:00<?, ?B/s]
100% 323k/323k [00:00<00:00, 105MB/s]
  0% 0.00/573k [00:00<?, ?B/s]
100% 573k/573k [00:00<00:00, 184MB/s]
  0% 0.00/325k [00:00<?, ?B/s]
100% 325k/325k [00:00<00:00, 104MB/s]
/content


In [4]:
!wget https://github.com/t-davidson/hate-speech-and-offensive-language/raw/master/data/labeled_data.p

!wget https://zenodo.org/record/3950379/files/extended_test-20200717T190516Z-001.zip
!unzip extended_test-20200717T190516Z-001.zip

!pip install pandarallel

--2021-04-03 20:39:20--  https://github.com/t-davidson/hate-speech-and-offensive-language/raw/master/data/labeled_data.p
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.p [following]
--2021-04-03 20:39:21--  https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.p
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3332233 (3.2M) [application/octet-stream]
Saving to: ‘labeled_data.p’

labeled_data.p      100%[===================>]   3.18M  20.5MB/s    in 0.2s    

2021-04-03 20:39

In [5]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from pandarallel import pandarallel

In [6]:
pandarallel.initialize()

INFO: Pandarallel will run on 2 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [7]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train_df = pd.read_csv('./kaggle/jigsaw/train.csv')
test_df = pd.read_csv('./kaggle/jigsaw/test.csv').merge(pd.read_csv('./kaggle/jigsaw/test_labels.csv'), on='id')
test_df.drop(test_df[test_df[label_cols].min(axis=1) == -1].index, inplace=True)

In [8]:
df = pd.concat([train_df, test_df], ignore_index=True)
df['toxic?'] = df.parallel_apply(lambda row: row[label_cols].any(), axis=1)
df['dataset'] = 0

In [9]:
df.drop(label_cols, inplace=True, axis=1)
df.drop('id', inplace=True, axis=1)
df.rename(columns={'comment_text': 'text'}, inplace=True)

In [10]:
df['toxic?'].mean()

0.10050592934882285

In [11]:
df

,text,toxic?,dataset
0,Explanation\nWhy the edits made under my usern...,0,0
1,D'aww! He matches this background colour I'm s...,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0
4,"You, sir, are my hero. Any chance you remember...",0,0
...,...,...,...
223544,":Jerome, I see you never got around to this…! ...",0,0
223545,==Lucky bastard== \n http://wikimediafoundatio...,0,0
223546,==shame on you all!!!== \n\n You want to speak...,0,0
223547,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1,0


In [12]:
df2 = pd.read_pickle('labeled_data.p')
df2.rename(columns={'tweet': 'text'}, inplace=True)
df2['toxic?'] = df2.parallel_apply(lambda row: int(row['class'] != 2), axis=1)
df2['dataset'] = 1
df2 = df2[['text', 'toxic?', 'dataset']]

In [13]:
df2['toxic?'].mean()

0.8320219505306057

In [14]:
df2

,text,toxic?,dataset
0,!!! RT @mayasolovely: As a woman you shouldn't...,0,1
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,1,1
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,1,1
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,1,1
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,1,1
...,...,...,...
25291,you's a muthaf***in lie &#8220;@LifeAsKing: @2...,1,1
25292,"you've gone and broke the wrong heart baby, an...",0,1
25294,young buck wanna eat!!.. dat nigguh like I ain...,1,1
25295,youu got wild bitches tellin you lies,1,1


In [15]:
df3 = pd.read_csv('onlineHarassmentDataset.tdf', sep='\t', header=0, encoding='latin-1')

In [16]:
df3.rename(columns={'Tweet': 'text'}, inplace=True)
df3['toxic?'] = df3.parallel_apply(lambda row: int(row['Code'] == 'H'), axis=1)
df3['dataset'] = 2
df3 = df3[['text', 'toxic?', 'dataset']]

In [17]:
df3['toxic?'].mean()

0.2595776031434185

In [18]:
# df3

In [19]:
train_df4 = pd.read_csv('./kaggle/impermium/train.csv', header=0)
test_df4 = pd.read_csv('./kaggle/impermium/test_with_solutions.csv', header=0)

In [20]:
df4 = pd.concat([train_df4, test_df4], ignore_index=True)
df4.rename(columns={'Comment': 'text', 'Insult': 'toxic?'}, inplace=True)
df4['dataset'] = 3
df4 = df4[['text', 'toxic?', 'dataset']]

In [21]:
df4['toxic?'].mean()

0.26417955717318775

In [22]:
df4

,text,toxic?,dataset
0,"""You fuck your dad.""",1,3
1,"""i really don't understand your point.\xa0 It ...",0,3
2,"""A\\xc2\\xa0majority of Canadians can and has ...",0,3
3,"""listen if you dont wanna get married to a man...",0,3
4,"""C\xe1c b\u1ea1n xu\u1ed1ng \u0111\u01b0\u1edd...",0,3
...,...,...,...
6589,"""Never really gave it much thought. I just fig...",0,3
6590,"""Nadie se salva de la regla 34 xd""",0,3
6591,"""Question: Are you a boy or a girl?""",0,3
6592,"""Leave your email or phone number and maybe yo...",1,3


In [23]:
df5_tweets = pd.read_csv('./extended_test/test_a_tweets_all.tsv', sep='\t', header=0)
df5_labels = pd.read_csv('./extended_test/test_a_labels_all.csv', names=['id', 'toxic?'])
df5 = df5_tweets.merge(df5_labels)

In [24]:
df5.rename(columns={'tweet': 'text'}, inplace=True)
df5['toxic?'] = df5.parallel_apply(lambda row: int(row['toxic?'] != 'NOT'), axis=1)
df5['dataset'] = 4
df5 = df5[['text', 'toxic?', 'dataset']]

In [25]:
df5['toxic?'].mean()

0.5009177373602536

In [26]:
df5

,text,toxic?,dataset
0,Has @USER quit? I've not heard of any #knifecr...,0,4
1,"In celebration of Emancipation Day, we urge yo...",0,4
2,@USER @USER It’d be a literal dream come true ...,0,4
3,Brilliant news to read that Hoggy has signed a...,0,4
4,@USER She speaks of the truth 😌,0,4
...,...,...,...
5988,@USER @USER Nothing about trump is human or no...,1,4
5989,@USER Oh shit. That sounds like a cool time tho,1,4
5990,@USER @USER I gotta say that shit to you one t...,1,4
5991,@USER tRUmp does everything he can to destroy ...,1,4


In [27]:
df = pd.concat([df, df2, df3, df4, df5], ignore_index=True)

In [28]:
df.shape, df['toxic?'].mean()

((281279, 3), 0.18884097284191143)

In [29]:
df.to_pickle('df.pkl')